In [1]:
import pandas as pd
import numpy as np
import datetime as dt


In [2]:
import keras

Using TensorFlow backend.


In [3]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras.layers import Masking

In [4]:
#read data from csv file
data=pd.read_csv('Book3.csv')
print(data.head())

                    Date  Hour  DryBulb  DewPnt  SYSLoad   NumDate
0  2004-01-01 00:00:00.0     1       37      25    12230  732000.0
1  2004-01-01 00:00:00.0     2       37      25    11534  732000.0
2  2004-01-01 00:00:00.0     3       39      24    11038  732000.0
3  2004-01-01 00:00:00.0     4       38      22    10777  732000.0
4  2004-01-01 00:00:00.0     5       37      20    10764  732000.0


In [5]:
#retrieve list of holidays
hdata=pd.read_excel('Holidays.xls')
hdata.dtypes

Date       datetime64[ns]
Holiday            object
dtype: object

In [6]:
holidays=hdata.Date
holidays.head()

0   2004-01-01
1   2004-01-19
2   2004-02-16
3   2004-05-31
4   2004-07-05
Name: Date, dtype: datetime64[ns]

In [7]:
#Convert dates to datetime data type
data['Date']=pd.to_datetime(data['Date'])
data.dtypes

Date       datetime64[ns]
Hour                int64
DryBulb             int64
DewPnt              int64
SYSLoad             int64
NumDate           float64
dtype: object

In [8]:
#get the day of the week for each date in the data
dayofweek=data.Date.dt.weekday
dayofweek.head()

0    3
1    3
2    3
3    3
4    3
Name: Date, dtype: int64

In [9]:
isworkday = np.in1d(dayofweek,[0,1,2,3,4]) & ~np.in1d(data['Date'],holidays)
isworkday[22]

False

In [10]:
prevdaysamehour=np.hstack(((np.ones(24)*-1),(data['SYSLoad'][0:-24])))
prevdaysamehour[0]

-1.0

In [11]:
prevweeksamehour=np.hstack(((np.ones(168)*-1),(data['SYSLoad'][0:-168])))
prevweeksamehour.shape

(52608,)

In [12]:
import scipy
import scipy.signal
#By using the lfilter method get the previous 24 hour averrage temperature 
prev24houravg = scipy.signal.lfilter(np.ones(24) / 24, 1, data['SYSLoad'])
prev24houravg.shape

(52608,)

In [13]:
#Put all the input data together into a single matrix
X = np.vstack((data['DryBulb'],data['DewPnt'],data['Hour'],dayofweek,isworkday,prevweeksamehour,prevdaysamehour,prev24houravg))
X.shape

(8, 52608)

In [14]:
X=np.transpose(X)
X.shape

(52608, 8)

In [1]:
numcols=X.shape[1]


NameError: name 'X' is not defined

In [16]:
#Take out the first 168 indexes because they have null values
X=X[168:,:]
data=data[168:]
print(X.shape)
print(data.shape)

(52440, 8)
(52440, 6)


In [19]:
#Find the training index that will be used to split the data into train and test
trainInd = data.Date < dt.datetime(2008, 1, 1) 
trainX = X[trainInd,:]
trainX.shape

(34896, 8)

In [20]:
#assign the training outputs
trainY=data.SYSLoad[trainInd]
trainY=trainY.as_matrix(columns=None)
trainY = trainY.reshape((trainY.shape[0], 1))
trainY.shape

C:\Users\afzal\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


(34896, 1)

In [31]:
#split the range for the validation data
validX=X[34896:43680]
validX.shape
validX[-1]

array([ 8.0000000e+00, -5.0000000e+00,  2.4000000e+01,  2.0000000e+00,
        1.0000000e+00,  1.3253000e+04,  1.3827000e+04,  1.6115375e+04])

In [32]:
validY=data.SYSLoad[34896:43680]
validY=validY.as_matrix(columns=None)
validY = validY.reshape((validY.shape[0], 1))
validY[-1]

C:\Users\afzal\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


array([15438], dtype=int64)

In [44]:
testInd = data.Date>= dt.datetime(2009, 1, 1)
testX=X[testInd,:]

array([2.7000000e+01, 2.4000000e+01, 2.4000000e+01, 3.0000000e+00,
       1.0000000e+00, 1.3803000e+04, 1.4466000e+04, 1.5745625e+04])

In [45]:
testY=data.SYSLoad[testInd]
testY=testY.as_matrix(columns=None)
testY = testY.reshape((testY.shape[0], 1))

C:\Users\afzal\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


array([13953], dtype=int64)

In [37]:
from keras.optimizers import SGD
model2 =Sequential()
model2.add(Dense(60,activation='relu' ,input_shape=(numcols,)))
model2.add(Dense(60,activation='relu'))
model2.add(Dense(1,activation='linear'))
model2.compile(optimizer='adam', loss='mean_absolute_error')

Instructions for updating:
Colocations handled automatically by placer.


In [38]:
import time
start_time = time.time()
history=model2.fit(trainX, trainY, batch_size=150, epochs=500,verbose=1)
time.time() - start_time

Instructions for updating:
Use tf.cast instead.
Epoch 1/500
34896/34896 [==============================] - 4s 126us/step - loss: 1437.5823
Epoch 2/500
34896/34896 [==============================] - 1s 16us/step - loss: 775.7344
Epoch 3/500
34896/34896 [==============================] - 1s 16us/step - loss: 755.5017
Epoch 4/500
34896/34896 [==============================] - 1s 16us/step - loss: 733.6473
Epoch 5/500
34896/34896 [==============================] - 1s 16us/step - loss: 728.6479
Epoch 6/500
34896/34896 [==============================] - 1s 16us/step - loss: 718.9223
Epoch 7/500
34896/34896 [==============================] - 1s 16us/step - loss: 713.6256
Epoch 8/500
34896/34896 [==============================] - 1s 15us/step - loss: 710.5816
Epoch 9/500
34896/34896 [==============================] - 1s 16us/step - loss: 705.9831
Epoch 10/500
34896/34896 [==============================] - 1s 18us/step - loss: 709.7126
Epoch 11/500
34896/34896 [==============================] -

Epoch 89/500
34896/34896 [==============================] - 1s 17us/step - loss: 631.3166
Epoch 90/500
34896/34896 [==============================] - 1s 18us/step - loss: 631.1633
Epoch 91/500
34896/34896 [==============================] - 1s 17us/step - loss: 632.1023
Epoch 92/500
34896/34896 [==============================] - 1s 16us/step - loss: 628.9337
Epoch 93/500
34896/34896 [==============================] - 1s 16us/step - loss: 621.3026
Epoch 94/500
34896/34896 [==============================] - 1s 16us/step - loss: 623.9249
Epoch 95/500
34896/34896 [==============================] - 1s 17us/step - loss: 620.8763
Epoch 96/500
34896/34896 [==============================] - 1s 16us/step - loss: 626.2482
Epoch 97/500
34896/34896 [==============================] - 1s 16us/step - loss: 620.9637
Epoch 98/500
34896/34896 [==============================] - 1s 16us/step - loss: 619.7080
Epoch 99/500
34896/34896 [==============================] - 1s 16us/step - loss: 617.1887
Epoch 100/

34896/34896 [==============================] - 1s 16us/step - loss: 565.3829
Epoch 180/500
34896/34896 [==============================] - 1s 16us/step - loss: 567.6879
Epoch 181/500
34896/34896 [==============================] - 1s 16us/step - loss: 556.3177
Epoch 182/500
34896/34896 [==============================] - 1s 16us/step - loss: 555.7222
Epoch 183/500
34896/34896 [==============================] - 1s 16us/step - loss: 556.7509: 0s - loss: 556.
Epoch 184/500
34896/34896 [==============================] - 1s 16us/step - loss: 560.3940
Epoch 185/500
34896/34896 [==============================] - 1s 17us/step - loss: 556.6374
Epoch 186/500
34896/34896 [==============================] - 1s 16us/step - loss: 552.7369
Epoch 187/500
34896/34896 [==============================] - 1s 17us/step - loss: 556.6111
Epoch 188/500
34896/34896 [==============================] - 1s 16us/step - loss: 573.4341
Epoch 189/500
34896/34896 [==============================] - 1s 16us/step - loss: 551.0

34896/34896 [==============================] - 1s 16us/step - loss: 514.3698
Epoch 359/500
34896/34896 [==============================] - 1s 16us/step - loss: 521.1044
Epoch 360/500
34896/34896 [==============================] - 1s 16us/step - loss: 513.9249
Epoch 361/500
34896/34896 [==============================] - 1s 17us/step - loss: 520.0229
Epoch 362/500
34896/34896 [==============================] - 1s 16us/step - loss: 515.2088
Epoch 363/500
34896/34896 [==============================] - 1s 16us/step - loss: 514.5308
Epoch 364/500
34896/34896 [==============================] - 1s 17us/step - loss: 524.6797
Epoch 365/500
34896/34896 [==============================] - 1s 16us/step - loss: 515.1491
Epoch 366/500
34896/34896 [==============================] - 1s 16us/step - loss: 512.3182
Epoch 367/500
34896/34896 [==============================] - 1s 16us/step - loss: 518.2856
Epoch 368/500
34896/34896 [==============================] - 1s 16us/step - loss: 515.2217
Epoch 369/500

Epoch 448/500
34896/34896 [==============================] - 1s 17us/step - loss: 509.6692
Epoch 449/500
34896/34896 [==============================] - 1s 16us/step - loss: 512.7595
Epoch 450/500
34896/34896 [==============================] - 1s 18us/step - loss: 506.7209
Epoch 451/500
34896/34896 [==============================] - 1s 16us/step - loss: 507.0260
Epoch 452/500
34896/34896 [==============================] - 1s 16us/step - loss: 513.1214
Epoch 453/500
34896/34896 [==============================] - 1s 16us/step - loss: 507.6010
Epoch 454/500
34896/34896 [==============================] - 1s 17us/step - loss: 511.1279
Epoch 455/500
34896/34896 [==============================] - 1s 16us/step - loss: 508.7082
Epoch 456/500
34896/34896 [==============================] - 1s 16us/step - loss: 511.1588
Epoch 457/500
34896/34896 [==============================] - 1s 16us/step - loss: 505.3634
Epoch 458/500
34896/34896 [==============================] - 1s 16us/step - loss: 508.4251

279.7330675125122

In [39]:
predictions= model2.predict(validX,verbose=2)
error=validY-predictions
#calculate mean absolute error for validation set
mae=np.mean(np.absolute(error))
mae

509.9725527459372

In [40]:
#calculate mean absolute percent error for validation set
errorpercent=np.absolute(error)/validY
mape=np.mean(errorpercent)*100
print(mape)

3.2977630798206317


In [46]:
testpredictions= model2.predict(testX,verbose=2)
#calculate means absolute error for test set
error=testY-testpredictions
mae=np.mean(np.absolute(error))
mae

451.72381419003284

In [47]:
#calulcate mean absolute percent error for test set
errorpercent=np.absolute(error)/testY
mape=np.mean(errorpercent)*100
print(mape)

3.062371216737978


In [117]:
import matplotlib.pyplot as plt
#plot the mean absolute percent error as a function of the number of epochs
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show() 

array([2.7000000e+01, 2.2000000e+01, 1.0000000e+00, 1.0000000e+00,
       0.0000000e+00, 1.2153000e+04, 1.2428000e+04, 1.4963375e+04])